In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap


%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp')
ovin_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_ovin_travel_behaviour.csv', index_col=0)
lms_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_lms_modal_split.csv', index_col=0)

In [ ]:
lms_orig = pd.read_csv(parent_dir + '\\Data\\New\\lms_modal_split_orig_abs.csv', index_col=0)
lms_dest = pd.read_csv(parent_dir + '\\Data\\New\\lms_modal_split_dest_abs.csv', index_col=0)

In [ ]:
lms_tot = lms_orig.iloc[:, 1:8].sum(axis=1) # Total trips for each zone

In [ ]:
lms_tb2 = lms_tb.iloc[:, 1:8].copy()
lms_tb2.iloc[:, 3] = lms_tb2.iloc[:, 3:5].sum(axis=1)
lms_tb2 = lms_tb2.drop(columns='Tram/Metro_o')
lms_tb2['Factor'] = lms_tot

In [ ]:
lms_tb2

### Get travel behaviour for each DU

In [ ]:
lms_du = np.zeros((6, 6))
ovin_du = np.zeros((6, 6))

for du in range(1, 7):

    zone_list = zones[zones.deg_urba == du].ZONE_ID # Get all zones with certain DU
    
    # Determine percentagese for LMS
    lms_sum = lms_orig[lms_orig.ZONE_ID.isin(zone_list)].sum()[1:] # Get total trips of each mode
    lms_tot = lms_sum.sum()

    p = lms_sum / lms_tot * 100
    p.iloc[3] = p.iloc[3:5].sum() # Combine btm
    p = p.iloc[np.r_[0:4, 5:7]] # Remove metro row

    lms_du[du - 1, :] = p

    # Determine percentages for OViN
    ovin_df = ovin_tb[ovin_tb.ZONE_ID.isin(zone_list)].iloc[:, np.r_[1:7, 29]] # Get right ovin data

    for i in range(6): # Loop over modes and calculate weighted average
        ovin_du[du - 1, i] = (ovin_df.iloc[:, i] * ovin_df.iloc[:, -1]).sum() / ovin_df.iloc[:, -1].sum()


In [ ]:
ovin_df

In [ ]:
ovin_tb.iloc[:, np.r_[1:7, 29]]

In [ ]:
def reindex_df(df, weight_col):
    """expand the dataframe to prepare for resampling
    result is 1 row per count per sample"""
    df = df.reindex(df.index.repeat(df[weight_col]))
    df.reset_index(drop=True, inplace=True)
    return(df)

## Plot

In [ ]:
du_list = np.arange(1, 7)

In [ ]:
np.max([lms_du.max(), ovin_du.max()])

(ovin_du - lms_du).max()

In [ ]:
mode_labels = ['Car driver', 'Car passenger', 'Train',
          'BTM', '(e)-Bike', 'Walking']

In [ ]:
f, ax = plt.subplots(3, 6)
f.set_figwidth(15)
f.set_figheight(15)


max_val = [np.max([lms_du.max(), ovin_du.max()]), 10, 80]
min_val = [0, -10, -50]
step = [5, 2, 10]

ax[1, 5].plot([], [], color="#D55E00", label='OViN', linewidth=5)
ax[1, 5].plot([], [], color="#0072B2", label='LMS', linewidth=5)
ax[1, 5].plot([], [], color="#009E73", label='Absolute difference OViN - LMS', linewidth=5)
ax[1, 5].plot([], [], color="#E69F00", label='Relative difference OViN - LMS', linewidth=5)

handles, labels = ax[1, 5].get_legend_handles_labels()


for i in range(3):

    for j in range(6):

        if i == 0:
            ax[0, j].bar(du_list - 0.2, ovin_du[:, j], color="#D55E00", width=0.35)
            
            ax[0, j].bar(du_list + 0.2, lms_du[:, j], color="#0072B2", width=0.35)
            
            ax[1, j].bar(du_list, (-ovin_du[:, j] + lms_du[:, j]), color="#009E73")

            ax[2, j].bar(du_list, (-ovin_du[:, j] + lms_du[:, j]) / ovin_du[:, j] * 100, color="#E69F00")


            # ax[0, j].set_title(f'{mode_labels[j]}')
            # ax[1, j].set_title(f'{mode_labels[j]}')
        
        # if i == 2:
        ax[i, j].set_xlabel('Degree of urbanisation')
        
        ax[i, j].set_title(f'{mode_labels[j]}')
        ax[i, j].set_axisbelow(True)
        ax[i, j].grid(axis='y')

        
        

        if i == 3:
            for k in range(6):
                ax[i, j].boxplot(reindex_df(ovin_tb[zones.deg_urba.isin([1 + k])].iloc[:, np.r_[1 + j, 29]].dropna(),
                                         'Factor_o').iloc[:, 0], positions=[k + 0.8], showfliers=False, 
                                         medianprops=dict(color="#D55E00", linewidth=2))
                

                ax[i, j].boxplot(reindex_df(lms_tb2[zones.deg_urba.isin([1 + k])].dropna(),
                                         'Factor').iloc[:, j], positions=[k + 1.2], showfliers=False, 
                                         medianprops=dict(color="#0072B2", linewidth=2))
                
                
                ax[i, j].set_ylim(0, 60)
        
        ax[i, j].set_yticks(np.arange(min_val[i], max_val[i] + step[i], step[i]))
        ax[i, j].set_xticks(np.arange(1, 7))


ax[0, 0].set_ylabel('Share of trips made by mode [%]')
ax[1, 0].set_ylabel('Absolute difference OViN - LMS [%-point]')
ax[2, 0].set_ylabel('Relative difference OViN - LMS [%]')




f.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.9, 1.0))
f.suptitle('(Difference in) modal split LMS and OViN \nfor degree of urbanisation of the origin zone',
           fontsize='xx-large');

In [ ]:
ovin_tb[zones.deg_urba.isin([1])].iloc[:, np.r_[1 + 0, 29]]

In [ ]:
ListedColormap(["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7", "#000000"])